In [1]:
import math
import random

In [2]:
import tensorflow 

In [3]:
import keras

In [4]:
import sys

In [5]:
import numpy as np
from collections import deque
import gym

In [6]:
env = gym.make('CartPole-v0')

for i_episode in range(20):
    observation = env.reset()
    for t in range(1000):
        env.render()
        #print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        


/opt/anaconda3/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [7]:
n_episodes = 1000
n_win_ticks = 195
max_env_steps = None

gamma = 1.0 
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.995
alpha = 0.01
alpha_decay = 0.01

batch_size = 64
monitor = False
quiet = False

memory = deque(maxlen = 100000)
env = gym.make('CartPole-v0')
if max_env_steps is not None: env.max_episode_steps = max_env_steps
    


In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam


In [9]:
model = Sequential()
model.add(Dense(24, input_dim=4, activation='relu'))
model.add(Dense(48, activation='relu'))
model.add(Dense(2, activation='relu'))
model.compile(loss='mse', optimizer=Adam(lr=alpha, decay=alpha_decay))

In [10]:
def remember(state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done))

def choose_action(state, epsilon):
    return env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(
        model.predict(state))

def get_epsilon(t):
    return max(epsilon_min, min(epsilon, 1.0 - math.log10((t + 1) * epsilon_decay)))

def preprocess_state(state):
    return np.reshape(state, [1, 4])

def replay(batch_size, epsilon):
    x_batch, y_batch = [], []
    minibatch = random.sample(memory, min(len(memory), batch_size))
    for state, action, reward, next_state, done in minibatch:
        y_target = model.predict(state)
        y_target[0][action] = reward if done else reward + gamma * np.max(model.predict(next_state)[0])
        x_batch.append(state[0])
        y_batch.append(y_target[0])

    model.fit(np.array(x_batch), np.array(y_batch), batch_size=len(x_batch), verbose=0)
    
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

In [11]:
def run():
    scores = deque(maxlen=100)

    for e in range(n_episodes):
        state = preprocess_state(env.reset())
        done = False
        i = 0
        while not done:
            action = choose_action(state, get_epsilon(e))
            next_state, reward, done, _ = env.step(action)
            env.render()
            next_state = preprocess_state(next_state)
            remember(state, action, reward, next_state, done)
            state = next_state
            i += 1

        scores.append(i)
        mean_score = np.mean(scores)
        if mean_score >= n_win_ticks and e >= 100:
            if not quiet: print('Ran {} episodes. Solved after {} trials'.format(e, e - 100))
            return e - 100
        if e % 100 == 0 and not quiet:
            print('[Episode {}] - Mean survival time over last 100 episodes was {} ticks.'.format(e, mean_score))
   
        replay(batch_size, epsilon)

    if not quiet: print('Did not solve after {} episodes'.format(e))
    return e

In [ ]:
import math
import random
import tensorflow 
import keras
import numpy as np
from collections import deque
import gym


n_episodes = 1000
n_win_ticks = 195
max_env_steps = None

gamma = 1.0 
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.995
alpha = 0.01
alpha_decay = 0.01

batch_size = 64
monitor = False
quiet = False

memory = deque(maxlen = 100000)
env = gym.make('CartPole-v0')
if max_env_steps is not None: env.max_episode_steps = max_env_steps
    
memory = deque(maxlen=100000)
env = gym.make('CartPole-v0')
if max_env_steps is not None: env.max_episode_steps = max_env_steps

from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

model = Sequential()
model.add(Dense(24, input_dim=4, activation='relu'))
model.add(Dense(48, activation='relu'))
model.add(Dense(2, activation='relu'))
model.compile(loss='mse', optimizer=Adam(lr=alpha, decay=alpha_decay))

def remember(state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done))

def choose_action(state, epsilon):
    return env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(
        model.predict(state))

def get_epsilon(t):
    return max(epsilon_min, min(epsilon, 1.0 - math.log10((t + 1) * epsilon_decay)))

def preprocess_state(state):
    return np.reshape(state, [1, 4])

def replay(batch_size, epsilon):
    x_batch, y_batch = [], []
    minibatch = random.sample(
        memory, min(len(memory), batch_size))
    for state, action, reward, next_state, done in minibatch:
        y_target = model.predict(state)
        y_target[0][action] = reward if done else reward + gamma * np.max(model.predict(next_state)[0])
        x_batch.append(state[0])
        y_batch.append(y_target[0])

    model.fit(np.array(x_batch), np.array(y_batch), batch_size=len(x_batch), verbose=0)
    
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay    
        
def run():
    scores = deque(maxlen=100)

    for e in range(n_episodes):
        state = preprocess_state(env.reset())
        done = False
        i = 0
        while not done:
            action = choose_action(state, get_epsilon(e))
            next_state, reward, done, _ = env.step(action)
            env.render()
            next_state = preprocess_state(next_state)
            remember(state, action, reward, next_state, done)
            state = next_state
            i += 1

        scores.append(i)
        mean_score = np.mean(scores)
        if mean_score >= n_win_ticks and e >= 100:
            if not quiet: print('Ran {} episodes. Solved after {} trials'.format(e, e - 100))
            return e - 100
        if e % 20 == 0 and not quiet:
            print('[Episode {}] - Mean survival time over last 100 episodes was {} ticks.'.format(e, mean_score))
   
        replay(batch_size, get_epsilon(e))

    if not quiet: print('Did not solve after {} episodes'.format(e))
    return e

run()

[Episode 0] - Mean survival time over last 100 episodes was 27.0 ticks.


Exception ignored on calling ctypes callback function: <function ObjCSubclass.method.<locals>.decorator.<locals>.objc_method at 0x13d80ad30>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/pyglet/libs/darwin/cocoapy/runtime.py", line 1121, in objc_method
    result = f(py_self, *args)
  File "/opt/anaconda3/lib/python3.8/site-packages/pyglet/window/cocoa/pyglet_window.py", line 64, in nextEventMatchingMask_untilDate_inMode_dequeue_
    if self.inLiveResize():
  File "/opt/anaconda3/lib/python3.8/site-packages/pyglet/libs/darwin/cocoapy/runtime.py", line 785, in __call__
    return self.method(self.objc_id, *args)
  File "/opt/anaconda3/lib/python3.8/site-packages/pyglet/libs/darwin/cocoapy/runtime.py", line 755, in __call__
    result = f(objc_id, self.selector, *args)
ctypes.ArgumentError: ("argument 1: <class 'RecursionError'>: maximum recursion depth exceeded while calling a Python object", "selector = b'inLiveResize'", "argtypes =[<class 'ctype

In [ ]:
import random
import gym
import math
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Training Parameters
n_episodes=1000
n_win_ticks=195
max_env_steps=None
gamma=1.0
epsilon=1.0
epsilon_min=0.01
epsilon_decay=0.995
alpha=0.01
alpha_decay=0.01
batch_size=64
monitor=False
quiet=False

# Environment Parameters
memory = deque(maxlen=100000)
env = gym.make('CartPole-v0')
if max_env_steps is not None: env.max_episode_steps = max_env_steps

from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Model Definition
model = Sequential()
model.add(Dense(24, input_dim=4, activation='relu'))
model.add(Dense(48, activation='relu'))
model.add(Dense(2, activation='relu'))
model.compile(loss='mse', optimizer=Adam(lr=alpha, decay=alpha_decay))

def remember(state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done))

def choose_action(state, epsilon):
    return env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(
        model.predict(state))

def get_epsilon(t):
    return max(epsilon_min, min(epsilon, 1.0 - math.log10((t + 1) * epsilon_decay)))

def preprocess_state(state):
    return np.reshape(state, [1, 4])

def replay(batch_size, epsilon):
    x_batch, y_batch = [], []
    minibatch = random.sample(
        memory, min(len(memory), batch_size))
    for state, action, reward, next_state, done in minibatch:
        y_target = model.predict(state)
        y_target[0][action] = reward if done else reward + gamma * np.max(model.predict(next_state)[0])
        x_batch.append(state[0])
        y_batch.append(y_target[0])

    model.fit(np.array(x_batch), np.array(y_batch), batch_size=len(x_batch), verbose=0)
    
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay    
        
def run():
    scores = deque(maxlen=100)

    for e in range(n_episodes):
        state = preprocess_state(env.reset())
        done = False
        i = 0
        while not done:
            action = choose_action(state, get_epsilon(e))
            next_state, reward, done, _ = env.step(action)
            env.render()
            next_state = preprocess_state(next_state)
            remember(state, action, reward, next_state, done)
            state = next_state
            i += 1

        scores.append(i)
        mean_score = np.mean(scores)
        if mean_score >= n_win_ticks and e >= 100:
            if not quiet: print('Ran {} episodes. Solved after {} trials'.format(e, e - 100))
            return e - 100
        if e % 20 == 0 and not quiet:
            print('[Episode {}] - Mean survival time over last 100 episodes was {} ticks.'.format(e, mean_score))
   
        replay(batch_size, get_epsilon(e))

    if not quiet: print('Did not solve after {} episodes'.format(e))
    return e

run()

[Episode 0] - Mean survival time over last 100 episodes was 19.0 ticks.


Exception ignored on calling ctypes callback function: <function ObjCSubclass.method.<locals>.decorator.<locals>.objc_method at 0x13cc6ce50>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/pyglet/libs/darwin/cocoapy/runtime.py", line 1121, in objc_method
    result = f(py_self, *args)
  File "/opt/anaconda3/lib/python3.8/site-packages/pyglet/window/cocoa/pyglet_window.py", line 64, in nextEventMatchingMask_untilDate_inMode_dequeue_
    if self.inLiveResize():
  File "/opt/anaconda3/lib/python3.8/site-packages/pyglet/libs/darwin/cocoapy/runtime.py", line 785, in __call__
    return self.method(self.objc_id, *args)
  File "/opt/anaconda3/lib/python3.8/site-packages/pyglet/libs/darwin/cocoapy/runtime.py", line 755, in __call__
    result = f(objc_id, self.selector, *args)
ctypes.ArgumentError: ("argument 1: <class 'RecursionError'>: maximum recursion depth exceeded while calling a Python object", "selector = b'inLiveResize'", "argtypes =[<class 'ctype